# Training fine flow prediction
Assuming source image $I_s$ and target image $I_t$ are already coarsely aligned, this notebook will try to predict a fine flow $F_{s\rightarrow t}$ between them. 

TODO describe objective functions used in this project

We assume you already have a folder called `workspace` that contains zipped dataset.

In [ ]:
%cd ../notebooks/workspace

This cell will run all stages.

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callback import EarlyStoppping
from pytorch_lightning.loggers import TensorBoardLogger

from ransacflow.data import MegaDepthDataModule
from ransacflow.train import RANSACFlowModel

if __name__ == "__main__":
    ransac_flow = RANSACFlowModel()

    # FIXME unify TB logging location and experiment name
    trainer = Trainer(
        logger=TensorBoardLogger('tb_logs', name='RANSAC-Flow'),
        callbacks=[EarlyStoppping(monitor="val_loss", min_delta=0.01, patience=3)]
    )
    trainer.fit(ransac_flow, MegaDepthDataModule)

## Stage 1
Only train the **reconstruction loss**.

All following command line interface are copied from the original implementation, temporarily.

In [ ]:
--nEpochs 200 
--lr 2e-4
--kernelSize 7 
--imgSize 224 
--batchSize 16 
--lambda-match 0.0 
--mu-cycle 0.0 
--grad 0.0  
--trainMode flow 
--margin 88 

## Stage 2
Train jointly the **reconstruction loss** and **cycle consistency of the flow**.

In [ ]:

--nEpochs 50 
--lr 2e-4 
--kernelSize 7 
--imgSize 224 
--batchSize 16 
--lambda-match 0.0 
--mu-cycle 1.0 
--grad 0.0  
--trainMode flow 
--margin 88 

## Stage 3
Train all three losses together: **reconstruction loss**, **cycle consistency of the flow**, and **matchability loss**.

In [ ]:
--nEpochs 50 
--lr 2e-4
--kernelSize 7 
--imgSize 224 
--batchSize 16 
--lambda-match 0.01 
--mu-cycle 1.0 
--grad 0.0  
--trainMode flow+match 
--margin 88 


## Stage 4
This additional stage fine tune on <TODO>, so the output image introduce less distortions.